Take a dictionary and format it an http json post

From https://stackoverflow.com/questions/28670835/python-socket-client-post-parameters
https://stackoverflow.com/questions/32436864/raw-post-request-with-json-in-body
https://stackoverflow.com/questions/5755507/creating-a-raw-http-request-with-sockets

In [ ]:
%websocketconnect 192.168.1.130:8266 --password horsetoe

In [ ]:
2+2

In [ ]:
#import upip

In [ ]:
#upip.install('urequests')

In [ ]:
sample_data = {'wind_direction_raw': 108,
               'rain_amount_raw': 0,
               'timestamp': (2020, 7, 26, 12, 2, 21, 6, 208),
               'elapsed_time': 6245,
               'wind_speed_raw': 5}

In [1]:
sample_data = tibble('wind_direction_raw' = 108, 
                     'rain_amount_raw' = 0, 
                     'timestamp' = paste(2020, 7, 26, 12, 2, 21, 6, 208, sep=", "), 
                     'elapsed_time' = 6245, 
                     'wind_speed_raw' = 5)

No device connected
  %serialconnect or websocketconnect to connect
  %lsmagic to list commands

In [ ]:
import urequests

def postData(data_dict, url="http://192.168.1.151:9494/station"):
    try:
        response = urequests.post(url,
            headers = {'Connection': 'Close', 'content-type': 'application/json'},
            json = data_dict)
    except Exception as e:
        print("Error sending packet: {}".format(e))
        return False
    return response

In [ ]:
response = postData(sample_data, "http://192.168.1.151:9494/station")

In [ ]:
if response:
    response.text
    response.status_code
    response.close()
else:
    print("message send failed")

In [ ]:
if not response:
    print("not!")

In [ ]:
def generateMessageID():
    mid = 0
    while True:
        yield mid
        mid = (mid + 1) & 0xff
        mid = mid if mid else 1
        
def processQueue(queue, message_id_generator, url="http://192.168.1.151:9494/station"):
    while len(queue) > 0:
        result = queue.popleft()
        
        if not "message_id" in result:
            result["message_id"] = next(message_id_generator)
        response = postData(result, url)
        
        if not response:
            print('POST failed')
            data_queue.appendleft(result) 
            
        elif response.status_code >= 300:
            print('POST failed with response {} and status {}'.format(response.text, response.status_code))
            data_queue.appendleft(result)
            response.close()
            
        else:
            print('POST succeded with response {} and status {}'.format(response.text, response.status_code))
            response.close()

In [16]:
from collections.deque import deque
from sensors import readData

get_ID = generateMessageID()
data_queue = deque()

data_queue.append(readData())

result = data_queue.popleft()
result["message_id"] = next(get_ID)


In [ ]:
result

In [17]:
test_queue = deque()

test_queue.append(readData())

result = test_queue.popleft()

if not "message_id" in result:
    result["message_id"] = "test"
    
result

{'elapsed_time': 7076, 'timestamp': (2020, 9, 13, 7, 56, 12, 6, 257), 'wind_direction_raw': 740, 'wind_speed_raw': 0, 'message_id': 'test', 'rain_amount_raw': 0}


In [18]:
data_queue.append(readData())
data_queue.append(readData())
data_queue.append(readData())

In [19]:
len(data_queue)

3


In [ ]:
data_queue.popleft()

In [ ]:
len(data_queue)

In [22]:
response = postData(data_queue.popleft(), "http://192.168.1.151:9494/station")

if not response:
    print('POST failed')
    data_queue.appendleft(result) 
    
elif response.status_code >= 300:
    print('POST failed with response {} with status {}'.format(response.text, response.status_code))
    data_queue.appendleft(result) 

else:
    print('POST response was {} with status {}'.format(response.text, response.status_code))
    try:
        response.close()
    except Exception as e:
        print(e)



*** OSError *** 


 'socket' object has no attribute 'read' 


*** Connection broken ***
You may need to reconnect

In [ ]:
2+2

In [ ]:
response = postData(data_queue.popleft(), "http://192.168.1.100")

if not response:
    print('POST failed')
    data_queue.appendleft(result) 
    
elif response.status_code >= 300:
    print('POST failed with response {} with status {}'.format(response.text, response.status_code))
    data_queue.appendleft(result) 

else:
    print('POST response was {} with status {}'.format(response.text, response.status_code))
    try:
        response.close()
    except Exception as e:
        print(e)

In [ ]:
response = postData(data_queue.popleft(), "http://192.168.1.151:9494/fakeEndpoint")

if not response:
    print('POST failed')
    data_queue.appendleft(result) 
    
elif response.status_code >= 300:
    print('POST failed with response {} with status {}'.format(response.text, response.status_code))
    data_queue.appendleft(result) 

else:
    print('POST response was {} with status {}'.format(response.text, response.status_code))
    try:
        response.close()
    except Exception as e:
        print(e)

In [ ]:
import os
os.listdir()

# All together now

In [14]:
import urequests

def generateMessageID():
    mid = 0
    while True:
        yield mid
        mid = (mid + 1) & 0xff
        mid = mid if mid else 1

def postData(data_dict, url="http://192.168.1.151:9494/station"):
    try:
        response = urequests.post(url,
            headers = {'Connection': 'Close', 'content-type': 'application/json'},
            json = data_dict)
    except Exception as e:
        print("Error sending packet: {}".format(e))
        return False
    return response

def processQueue(queue, message_id_generator, url="http://192.168.1.151:9494/station"):
    failures = 0
    while (len(queue) > 0) and (failures < 5):
        print('queue has {} items and {} failures'.format(len(queue), failures))
        result = queue.popleft()
        
        if not "message_id" in result:
            result["message_id"] = next(message_id_generator)
            print("attaching message id {}".format(result["message_id"]))
        response = postData(result, url)
        
        if not response:
            print('POST failed')
            data_queue.appendleft(result) 
            failures = failures + 1
            
        elif response.status_code >= 300:
            print('POST failed with response {} and status {}'.format(response.text, response.status_code))
            data_queue.appendleft(result)
            response.close()
            failures = failures + 1
            
        else:
            print('POST succeded with response {} and status {}'.format(response.text, response.status_code))
            response.close()
            failures = 0

In [15]:
2+2


4


In [26]:
netclient_script="""import urequests

def generateMessageID():
    mid = 0
    while True:
        yield mid
        mid = (mid + 1) & 0xff
        mid = mid if mid else 1

def postData(data_dict, url="http://192.168.1.151:9494/station"):
    try:
        response = urequests.post(url,
            headers = {'Connection': 'Close', 'content-type': 'application/json'},
            json = data_dict)
    except Exception as e:
        print("Error sending packet: {}".format(e))
        return False
    return response
   
def processQueue(queue, message_id_generator, url="http://192.168.1.151:9494/station"):
    failures = 0
    while len(queue) > 0 and failures < 5:
        result = queue.popleft()
        
        if not "message_id" in result:
            result["message_id"] = next(message_id_generator)
        response = postData(result, url)
        
        if not response:
            print('POST failed')
            data_queue.appendleft(result) 
            failures = failures + 1
            
        elif response.status_code >= 300:
            print('POST failed with response {} and status {}'.format(response.text, response.status_code))
            data_queue.appendleft(result)
            response.close()
            failures = failures + 1
            
        else:
            print('POST succeded with response {} and status {}'.format(response.text, response.status_code))
            response.close()
            failures = 0

"""

In [27]:
os.listdir()

['boot.py', 'credentials.json', 'credentials.txt', 'lib', 'send_data.py', 'sensors.py', 'webrepl_cfg.py', 'wifi.py']


In [28]:
os.remove('send_data.py')

In [29]:
with open('send_data.py') as f:
    print(f.read())

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: [Errno 2] ENOENT


In [30]:
with open('send_data.txt', 'w') as f:
    f.write(netclient_script)

1444


### Double check the write

In [31]:
with open('send_data.txt') as f:
    print(f.read())

import urequests

def generateMessageID():
    mid = 0
    while True:
        yield mid
        mid = (mid + 1) & 0xff
        mid = mid if mid else 1

def postData(data_dict, url="http://192.168.1.151:9494/station"):
    try:
        response = urequests.post(url,
            headers = {'Connection': 'Close', 'content-type': 'application/json'},
            json = data_dict)
    except Exception as e:
        print("Error sending packet: {}".format(e))
        return False
    return response
   
def processQueue(queue, message_id_generator, url="http://192.168.1.151:9494/station"):
    failures = 0
    while len(queue) > 0 and failures < 5:
        result = queue.popleft()
        
        if not "message_id" in result:
            result["message_id"] = next(message_id_gerator)
(        response = postData(result, url)
        
        if not response:
           print('POST failed')
*          data_queue.appendleft(result) #          failures = failures + 1
          
) 

## Move temp file to `send_data.py`

In [32]:
os.rename('send_data.txt', 'send_data.py')

# TODO

How to log status codes and errors to detect errors

How to save data if no network connection, before restart

In [33]:
%disconnect

Device esp8266 disconnected.